In [1]:
from PIL import Image
import os
from pathlib import Path

def compress_png_lossy(source_folder, quality=75, max_size=None):
    """
    使用有损压缩方式压缩PNG图片
    
    参数:
    source_folder: 源文件夹路径
    quality: 图片质量 (1-100, 推荐70-85)
    max_size: 可选，限制图片最大尺寸 (width, height)
    """
    source_path = Path(source_folder)
    
    if not source_path.exists():
        print(f"文件夹不存在: {source_folder}")
        return
    
    # 统计信息
    total_original_size = 0
    total_compressed_size = 0
    processed_count = 0
    
    # 遍历所有子文件夹和文件
    for root, dirs, files in os.walk(source_path):
        for filename in files:
            if filename.lower().endswith('.png'):
                file_path = Path(root) / filename
                
                try:
                    # 获取原始文件大小
                    original_size = file_path.stat().st_size
                    
                    # 打开图片
                    with Image.open(file_path) as img:
                        # 保存原始模式
                        original_mode = img.mode
                        
                        # 如果需要调整尺寸
                        if max_size and (img.width > max_size[0] or img.height > max_size[1]):
                            img.thumbnail(max_size, Image.Resampling.LANCZOS)
                            print(f"  调整尺寸: {img.size}")
                        
                        # 根据图片模式选择压缩策略
                        if original_mode in ('RGBA', 'LA', 'P'):
                            # 有透明通道，使用PNG的调色板模式压缩
                            if original_mode == 'P':
                                img = img.convert('RGBA')
                            
                            # 量化颜色（减少颜色数量）
                            img = img.quantize(colors=256, method=2)
                            img.save(file_path, 'PNG', optimize=True, compress_level=9)
                        else:
                            # 无透明通道，先转JPEG再转回PNG（有损但压缩率高）
                            if img.mode != 'RGB':
                                img = img.convert('RGB')
                            
                            # 临时保存为JPEG进行有损压缩
                            temp_path = file_path.with_suffix('.tmp.jpg')
                            img.save(temp_path, 'JPEG', quality=quality, optimize=True)
                            
                            # 读取JPEG并保存回PNG
                            with Image.open(temp_path) as compressed_img:
                                compressed_img.save(file_path, 'PNG', optimize=True, compress_level=9)
                            
                            # 删除临时文件
                            temp_path.unlink()
                    
                    # 获取压缩后文件大小
                    compressed_size = file_path.stat().st_size
                    
                    total_original_size += original_size
                    total_compressed_size += compressed_size
                    processed_count += 1
                    
                    # 显示处理进度
                    compression_ratio = (1 - compressed_size / original_size) * 100 if original_size > 0 else 0
                    print(f"✓ {file_path.relative_to(source_path)}")
                    print(f"  {original_size/1024:.2f}KB -> {compressed_size/1024:.2f}KB "
                          f"(压缩 {compression_ratio:.1f}%)")
                    
                except Exception as e:
                    print(f"✗ 处理失败 {file_path.relative_to(source_path)}: {e}")
    
    # 显示总结
    print("\n" + "="*60)
    print(f"压缩完成!")
    print(f"处理文件数: {processed_count}")
    print(f"原始总大小: {total_original_size/1024/1024:.2f}MB")
    print(f"压缩后总大小: {total_compressed_size/1024/1024:.2f}MB")
    if total_original_size > 0:
        total_compression = (1 - total_compressed_size / total_original_size) * 100
        print(f"总压缩率: {total_compression:.1f}%")
        print(f"节省空间: {(total_original_size - total_compressed_size)/1024/1024:.2f}MB")

# 有损压缩，质量30，限制尺寸
compress_png_lossy('images', quality=30, max_size=(1920, 1080))

# 或者不限制尺寸，只进行质量压缩
# compress_png_lossy('images', quality=80)

✓ 乐舞\乐名\九招.png
  1513.63KB -> 1216.80KB (压缩 19.6%)
✓ 乐舞\乐名\九歌.png
  1683.10KB -> 1279.87KB (压缩 24.0%)
✓ 乐舞\乐名\九辯.png
  1549.58KB -> 1121.08KB (压缩 27.7%)
✓ 乐舞\乐名\樂風.png
  1561.03KB -> 1290.67KB (压缩 17.3%)
✓ 乐舞\乐器\梧.png
  1517.08KB -> 982.90KB (压缩 35.2%)
✓ 乐舞\乐器\琴.png
  1328.68KB -> 909.47KB (压缩 31.6%)
✓ 乐舞\乐器\瑟.png
  1574.39KB -> 1228.56KB (压缩 22.0%)
✓ 乐舞\乐器\磬.png
  1771.30KB -> 1395.23KB (压缩 21.2%)
✓ 乐舞\乐器\鍾.png
  1303.57KB -> 876.44KB (压缩 32.8%)
✓ 乐舞\舞名\九代.png
  1523.94KB -> 1169.37KB (压缩 23.3%)
✓ 乐舞\舞名\二人儛.png
  1642.28KB -> 1402.33KB (压缩 14.6%)
✓ 乐舞\舞名\儛.png
  1291.20KB -> 617.83KB (压缩 52.2%)
✓ 乐舞\舞名\干儛.png
  1651.99KB -> 1369.94KB (压缩 17.1%)
✓ 乐舞\舞名\璆冕舞.png
  1627.51KB -> 1158.09KB (压缩 28.8%)
✓ 人物\人名\一臂民.png
  1864.85KB -> 1632.15KB (压缩 12.5%)
✓ 人物\人名\三身.png
  1632.42KB -> 1327.69KB (压缩 18.7%)
✓ 人物\人名\三靣之人.png
  1461.04KB -> 914.39KB (压缩 37.4%)
✓ 人物\人名\三頭人.png
  1772.18KB -> 1360.84KB (压缩 23.2%)
✓ 人物\人名\不死民.png
  1792.20KB -> 1541.46KB (压缩 14.0%)
✓ 人物\人名\中容.png
  1771.91KB -> 1464.